# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# First I am going to read in the Dataframe that I created in the WeatherPy file
cities_file_location = "../Weather/output_data/cities.csv"

weather_df = pd.read_csv(cities_file_location)
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Country,Temperature (F),Humidity %,Cloudiness %,Wind Speed (mph)
0,0,Coyhaique,-45.5752,-72.0662,CL,40.95,65,75,3.44
1,1,Fuerte Olimpo,-21.0415,-57.8738,PY,85.53,25,2,5.30
2,2,Punta Arenas,-53.1500,-70.9167,CL,42.12,81,0,13.80
3,3,Barrow,71.2906,-156.7887,US,48.22,93,90,6.91
4,4,Bredasdorp,-34.5322,20.0403,ZA,58.32,47,100,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Here I am configuring gmaps 
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# I am assigning the weight of the heat map to the humidity % of each city so I need to read this in from the dataframe
rating = weather_df["Humidity %"].astype(float)


# Create initial figure with gmaps
fig = gmaps.figure(center = (30,0), zoom_level=1.7)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, dissipating=False, point_radius=3.3, max_intensity=150)
# Add heat layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# drop null rows
weather_df = weather_df.dropna()

In [6]:
# Filter out rows that are too hot or too cold 
# Only temps from 70 to 80 will still be in dataframe
not_too_cold_df = weather_df.loc[weather_df["Temperature (F)"]>=70]
ideal_temp_df = not_too_cold_df.loc[not_too_cold_df["Temperature (F)"]<=80]


# From this ideal temperature dataframe I am now going to filter out rows where it is too windy
# I will define too windy as any city that has wind speeds of over 10 mph 
ideal_wind_df = ideal_temp_df.loc[ideal_temp_df["Wind Speed (mph)"]<=10]

ideal_cloud_df = ideal_wind_df.loc[ideal_wind_df["Cloudiness %"]<=50]
ideal_weather_df = ideal_cloud_df.loc[ideal_cloud_df["Humidity %"]<=70]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
# Create a series of latitudes and longitudes and convert them to lists
lat = ideal_weather_df["Latitude"]
lat = lat.tolist()
longt = ideal_weather_df["Longitude"]
longt = longt.tolist()
print(lat[3])
print(longt[3])

37.2687
28.2167


In [16]:
# Set up parameters 
target_search = "Hotel"
target_radius = 5000
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up blank list 
hotels = []

target_coordinates = str(lat[3]) +", "+ str(longt[3])
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key,
    "rankBy": "DISTANCE"
    }
response = requests.get(base_url, params=params)
hotel_data = response.json()
pprint(hotel_data)
print(hotel_data['results'][0]['name'])

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


IndexError: list index out of range

In [13]:
# Set up parameters 
target_search = "Hotel"
target_radius = 5000
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up blank list 
hotels = []

for x in range(len(ideal_weather_df)):
    target_coordinates = str(lat[x]) +", "+ str(longt[x])
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key,
        "rankBy": "DISTANCE"
    }
    response = requests.get(base_url, params=params)
    hotel_data = response.json()
    print(x)
    print(hotel_data['results'][0]['name'])
    hotels.append(hotel_data['results'][0]['name'])

0
Hotel e Restaurante Viva Maria
1
Zolotoy Fazan
2
Grand Hotel of the Angels
3


IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
